In [272]:
import pandas as pd
import sklearn
import numpy as np

from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans

In [273]:
cf = pd.read_csv("notebooks/county_facts_with_2012_winner.csv")

# get array of fipscodes to later assign as labels by county
fipscodes = cf["fips"].values
winner = cf["winner"].values

In [274]:
# list of attributes that are the most important according to weka
weka_list = ["SEX255214", "RHI125214", "RHI225214","RHI625214", "RHI725214", 
            "RHI825214", "POP815213", "EDU635213", "LFE305213","HSG495213",
            "HSD310213","INC910213","PVY020213","SBO415207", "RTN131207","LND110210", "POP060210"]

In [275]:
# remember the state abbreviations for later (since we need only numeric values)
y = cf["state_abbreviation"].values
cf = cf.drop(['state_abbreviation'], axis=1)

# our table is now only numeric values
cf = cf[weka_list]

# ???
newcf_norm = (cf - cf.mean()) / (cf.max() - cf.min())
newcf_norm = newcf_norm.div(newcf_norm.sum(axis=1), axis=0)

X = cf.as_matrix() # take our dataframe as numpy matrix
kmeans_model = KMeans(n_clusters=4, random_state=1).fit(X) # build model
labels = kmeans_model.labels_ # our cluster labels
clusters = pd.DataFrame(labels, fipscodes) # our reformed dataset, matching the clusters to the fips codes
clusters = clusters.reset_index() 
clusters.columns = ['fips', 'cluster']
clusters # two columns, [fips, cluster]

for i in range(0, 4): # print the size of each cluster
    print len(clusters.loc[clusters['cluster'] == i])

242
1747
56
912


In [311]:
cf["fips"] = fipscodes
cf["cluster"] = clusters['cluster']
cf["winner"] = winner

fi = pd.DataFrame() # create empty dataframe

for i in range(0,4):
    clustered_facts_grouped = cf[cf["cluster"]==i] # groupby cluster
    
#     newcf_norm = (cf - cf.mean()) / (cf.max() - cf.min())
#     newcf_norm = newcf_norm.div(newcf_norm.sum(axis=1), axis=0)

    y = clustered_facts_grouped["winner"]
    clustered_facts_grouped = clustered_facts_grouped.drop(['fips', 'cluster', 'winner'], axis=1)

    X = clustered_facts_grouped.as_matrix()
        
    model = ExtraTreesClassifier() 
    model.fit(X, y) # find which features, in this cluster, were most beneficial to determining the winner

    z = pd.Series(model.feature_importances_)
    fi = fi.append(z, ignore_index=True)
    
fi.columns = weka_list
for index, row in fi.iterrows():
    print row.sort

<bound method Series.sort of SEX255214    0.085416
RHI125214    0.061982
RHI225214    0.060368
RHI625214    0.079238
RHI725214    0.042810
RHI825214    0.052548
POP815213    0.034415
EDU635213    0.090666
LFE305213    0.083731
HSG495213    0.045280
HSD310213    0.098120
INC910213    0.064470
PVY020213    0.049343
SBO415207    0.026649
RTN131207    0.026774
LND110210    0.057848
POP060210    0.040341
Name: 0, dtype: float64>
<bound method Series.sort of SEX255214    0.041558
RHI125214    0.074233
RHI225214    0.077382
RHI625214    0.066464
RHI725214    0.074798
RHI825214    0.054477
POP815213    0.059325
EDU635213    0.070821
LFE305213    0.055889
HSG495213    0.058816
HSD310213    0.054814
INC910213    0.057203
PVY020213    0.052365
SBO415207    0.015708
RTN131207    0.051399
LND110210    0.072865
POP060210    0.061883
Name: 1, dtype: float64>
<bound method Series.sort of SEX255214    0.050894
RHI125214    0.077498
RHI225214    0.119959
RHI625214    0.059790
RHI725214    0.043261
RHI82